In [0]:

## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

In [0]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc2
/device:GPU:0
Num GPUs Available:  1


In [0]:
from tensorflow.keras.callbacks import Callback

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")
!ls
import os
os.chdir("My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
'My Drive'


In [0]:
np.random.seed(12227)

In [0]:
def DataPreparation(data_input_file):
  print('David M Burns et al. 2018 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  # classes_number = Y.shape[1]
  # Y = np.argmax(Y, axis=1)
  return X,Y,folds

In [0]:
x,y,f=DataPreparation('/content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz')

Jian Sun et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz


In [0]:
x=x[0,0,:,:]

In [0]:
y.shape

(3871, 12)

In [0]:
x.shape

(160, 3)

In [0]:
x

array([[30., 40., 51.],
       [30., 41., 51.],
       [31., 41., 51.],
       [29., 42., 51.],
       [30., 43., 52.],
       [30., 40., 52.],
       [31., 41., 52.],
       [32., 39., 52.],
       [29., 38., 52.],
       [29., 38., 50.],
       [28., 40., 50.],
       [31., 38., 51.],
       [30., 39., 51.],
       [30., 39., 50.],
       [31., 39., 51.],
       [30., 38., 52.],
       [29., 39., 53.],
       [31., 38., 52.],
       [29., 39., 53.],
       [31., 37., 53.],
       [29., 38., 53.],
       [29., 42., 51.],
       [31., 38., 53.],
       [30., 37., 53.],
       [31., 42., 52.],
       [30., 39., 51.],
       [29., 38., 53.],
       [29., 41., 51.],
       [27., 40., 50.],
       [27., 41., 49.],
       [26., 42., 48.],
       [25., 46., 49.],
       [25., 42., 49.],
       [24., 42., 49.],
       [25., 41., 48.],
       [24., 42., 49.],
       [24., 42., 50.],
       [23., 43., 50.],
       [22., 40., 48.],
       [22., 43., 48.],
       [22., 42., 48.],
       [17., 42.

In [0]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [0]:
def Run_David_Burns(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)
  # idx = [3, 5, 8, 11, 14, 17, 20] #For MHEALTH
  # idx = [1, 4, 7, 10, 13, 14, 17, 20, 23, 26, 27, 30, 33, 36]#For PAMAP2P
  # idx = [val for val in idx for _ in (0, 1)] #Vertical Kernel-1
  # X = zero_padding_MHEALTH(X)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [0]:
# def build_model(row,col,num_classes):

  
 
#   layers = [
#        tf.keras.layers.Conv1D(filters=512,kernel_size=3,strides=1),
#        tf.keras.layers.MaxPool1D(pool_size=2,strides = 1),
#         tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1),
#        tf.keras.layers.MaxPool1D(pool_size=2,strides = 1),
#       tf.keras.layers.Conv1D(filters=64,kernel_size=3,strides=1),
#        tf.keras.layers.MaxPool1D(pool_size=2,strides = 1),
#         tf.keras.layers.Conv1D(filters=32,kernel_size=3,strides=1),
#        tf.keras.layers.MaxPool1D(pool_size=2,strides = 1),
#       #  tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
#       #  tf.keras.layers.Reshape((col,row)),
     
        
#         tf.keras.layers.LSTM(128,return_sequences=True),
#         tf.nn.dropout(0.3),
#         tf.keras.layers.Dense(1000),
             
                  

       

       
        
       
#         tf.keras.layers.Dense(num_classes, activation = 'softmax')
#     ]
    
#   model = tf.keras.Sequential(layers)
#   return model

In [0]:
from tensorflow.keras.layers import Lambda

In [0]:
def fcn(input_shape, num_classes, conv_layers=({'f': 128, 'k': 8, 's': 1}, {'f': 256, 'k': 5, 's': 1}, {'f': 128, 'k': 3, 's': 1}),
        dropout=0.3, normalize=True, embedding_size=None):
    """
    Creates fully convolutional neural (FCN) network architecture described in: https://arxiv.org/abs/2001.05517
    :param input_shape: tuple (2)
        segment shape (width, n_channels)
    :param conv_layers: tuple of dicts
        describe conv layers with f: filters, k: kernel size, s: stride
    :param dropout: float
        dropout ratio applied at each layer
    :param normalize: bool
        apply l2 normalization
    :param embedding_size: integer, optional
        defines embedding size (number of filters for last CNN layer)
    :return: keras model
        the fcn model
    """

    input_layer = tf.keras.Input(shape=input_shape)
    layer = input_layer

    if embedding_size:
        conv_layers[-1]['f'] = embedding_size

    for lp in conv_layers:
        layer = tf.keras.layers.Conv1D(filters=lp['f'], kernel_size=lp['k'], strides=lp['s'], padding='same')(layer)
        layer = tf.keras.layers.BatchNormalization()(layer)
        layer = tf.keras.layers.Activation('relu')(layer)

        if dropout:
            layer = tf.keras.layers.Dropout(rate=dropout)(layer)

    layer = tf.keras.layers.GlobalAveragePooling1D()(layer)

    if normalize:
        layer = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=1))(layer)
    layer= tf.keras.layers.Dense(num_classes, activation = 'softmax')   (layer) 

    return tf.keras.Model(inputs=input_layer, outputs=layer)

In [0]:
# def build_model(row,col,num_classes):

  
 
#   layers = [
#        tf.keras.layers.Conv1D(filters=128,kernel_size=8,strides=1),
#        tf.keras.layers.BatchNormalization(),
#        tf.keras.layers.Activation('relu'),
#        tf.keras.layers.Dropout(0.3),

#         tf.keras.layers.Conv1D(filters=256,kernel_size=5,strides=1),
#        tf.keras.layers.BatchNormalization(),
#        tf.keras.layers.Activation('relu'),
#        tf.keras.layers.Dropout(0.3),

#        tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1),
#        tf.keras.layers.BatchNormalization(),
#        tf.keras.layers.Activation('relu'),
#        tf.keras.layers.Dropout(0.3),
#        tf.keras.layers.GlobalAveragePooling1D(),
#        tf.keras.layers.l2_normalize(),


     
        
       
             
                  

       

       
        
       
#         tf.keras.layers.Dense(num_classes, activation = 'softmax')
#     ]
    
#   model = tf.keras.Sequential(layers)
#   return model

In [0]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _,img_rows, img_cols = X.shape
  print(X.shape)
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    model=fcn((img_rows,img_cols),n_class)
    print(model.summary)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='SGD')
    model.fit(X_train, y[train_idx], batch_size=64, epochs=150)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [0]:
tf.keras.backend.set_image_data_format('channels_first')

In [0]:
Run_David_Burns('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

David M Burns et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz
(2555, 250, 23)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7f1cc5b79d30>>
Epoch 1/150
36/36 [==============================] - 1s 15ms/step - loss: 2.4387 - accuracy: 0.1230
Epoch 2/150
36/36 [==============================] - 0s 13ms/step - loss: 2.3535 - accuracy: 0.2135
Epoch 3/150
36/36 [==============================] - 0s 13ms/step - loss: 2.2956 - accuracy: 0.2957
Epoch 4/150
36/36 [==============================] - 0s 14ms/step - loss: 2.2502 - accuracy: 0.3357
Epoch 5/150
36/36 [==============================] - 0s 13ms/step - loss: 2.2135 - accuracy: 0.4157
Epoch 6/150
36/36 [==============================] - 0s 14ms/step - loss: 2.1835 - accuracy: 0.5770
Epoch 7/150
36/36 [==============================] - 0s 13ms/step - loss: 2.1564 - accuracy: 0.6530
Epoch 8/150
36/36 [==============================] - 0s 14ms/step - loss: 2.1323 

In [0]:
Run_David_Burns('/content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz')

David M Burns et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz
(9824, 500, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7f1cc20bde48>>
Epoch 1/150
140/140 [==============================] - 3s 21ms/step - loss: 2.3466 - accuracy: 0.2718
Epoch 2/150
140/140 [==============================] - 3s 20ms/step - loss: 2.2282 - accuracy: 0.2806
Epoch 3/150
140/140 [==============================] - 3s 20ms/step - loss: 2.1628 - accuracy: 0.2815
Epoch 4/150
140/140 [==============================] - 3s 20ms/step - loss: 2.1135 - accuracy: 0.2845
Epoch 5/150
140/140 [==============================] - 3s 20ms/step - loss: 2.0729 - accuracy: 0.2868
Epoch 6/150
140/140 [==============================] - 3s 20ms/step - loss: 2.0370 - accuracy: 0.2987
Epoch 7/150
140/140 [==============================] - 3s 20ms/step - loss: 2.0045 - accuracy: 0.3164
Epoch 8/150
140/140 [==============================] - 3s 21ms/step -

In [0]:
Run_David_Burns('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz')

David M Burns et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz
(3771, 50, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7f32603a18d0>>
Epoch 1/150
52/52 [==============================] - 0s 8ms/step - loss: 3.0358 - accuracy: 0.0600
Epoch 2/150
52/52 [==============================] - 0s 6ms/step - loss: 3.0229 - accuracy: 0.0750
Epoch 3/150
52/52 [==============================] - 0s 6ms/step - loss: 3.0104 - accuracy: 0.0820
Epoch 4/150
52/52 [==============================] - 0s 6ms/step - loss: 3.0014 - accuracy: 0.0917
Epoch 5/150
52/52 [==============================] - 0s 6ms/step - loss: 2.9910 - accuracy: 0.1073
Epoch 6/150
52/52 [==============================] - 0s 6ms/step - loss: 2.9836 - accuracy: 0.1064
Epoch 7/150
52/52 [==============================] - 0s 6ms/step - loss: 2.9759 - accuracy: 0.1073
Epoch 8/150
52/52 [==============================] - 0s 6ms/step - loss: 2.9677 - acc

In [0]:
Run_David_Burns('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz')

David M Burns et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz
(1137, 50, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7f324e4b8b00>>
Epoch 1/150
16/16 [==============================] - 0s 10ms/step - loss: 1.7819 - accuracy: 0.2229
Epoch 2/150
16/16 [==============================] - 0s 6ms/step - loss: 1.7466 - accuracy: 0.2482
Epoch 3/150
16/16 [==============================] - 0s 5ms/step - loss: 1.7134 - accuracy: 0.2786
Epoch 4/150
16/16 [==============================] - 0s 5ms/step - loss: 1.6872 - accuracy: 0.2888
Epoch 5/150
16/16 [==============================] - 0s 5ms/step - loss: 1.6663 - accuracy: 0.2999
Epoch 6/150
16/16 [==============================] - 0s 6ms/step - loss: 1.6513 - accuracy: 0.3364
Epoch 7/150
16/16 [==============================] - 0s 6ms/step - loss: 1.6341 - accuracy: 0.3921
Epoch 8/150
16/16 [==============================] - 0s 5ms/step - loss: 1.6208 - ac

In [0]:
Run_David_Burns('/content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz')

David M Burns et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz
(3871, 160, 3)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7f322e248f28>>
Epoch 1/150
35/35 [==============================] - 0s 10ms/step - loss: 2.3756 - accuracy: 0.3262
Epoch 2/150
35/35 [==============================] - 0s 8ms/step - loss: 2.2358 - accuracy: 0.4404
Epoch 3/150
35/35 [==============================] - 0s 8ms/step - loss: 2.1539 - accuracy: 0.4884
Epoch 4/150
35/35 [==============================] - 0s 8ms/step - loss: 2.0981 - accuracy: 0.4915
Epoch 5/150
35/35 [==============================] - 0s 8ms/step - loss: 2.0482 - accuracy: 0.4810
Epoch 6/150
35/35 [==============================] - 0s 8ms/step - loss: 2.0106 - accuracy: 0.4879
Epoch 7/150
35/35 [==============================] - 0s 8ms/step - loss: 1.9768 - accuracy: 0.4920
Epoch 8/150
35/35 [==============================] - 0s 8ms/step - loss: 1.9447 - accuracy

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7f317ab7ac88>>
Epoch 1/150
59/59 [==============================] - 1s 10ms/step - loss: 2.3871 - accuracy: 0.3128
Epoch 2/150
59/59 [==============================] - 0s 8ms/step - loss: 2.2352 - accuracy: 0.4113
Epoch 3/150
59/59 [==============================] - 0s 8ms/step - loss: 2.1497 - accuracy: 0.4635
Epoch 4/150
59/59 [==============================] - 1s 9ms/step - loss: 2.0932 - accuracy: 0.5072
Epoch 5/150
59/59 [==============================] - 0s 8ms/step - loss: 2.0481 - accuracy: 0.5224
Epoch 6/150
59/59 [==============================] - 0s 8ms/step - loss: 2.0130 - accuracy: 0.5288
Epoch 7/150
59/59 [==============================] - 1s 9ms/step - loss: 1.9832 - accuracy: 0.5264
Epoch 8/150
59/59 [==============================] - 1s 8ms/step - loss: 1.9572 - accuracy: 0.5349
Epoch 9/150
59/59 [==============================] - 0s 8ms/step - loss: 1.9351 - accuracy: 0.5370
E

In [0]:
Run_David_Burns('data/LOSO/WISDM.npz')

David M Burns et al. 2018 data/LOSO/WISDM.npz
(20846, 100, 3)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7ff040598978>>
Epoch 1/150
316/316 [==============================] - 2s 7ms/step - loss: 1.4199 - accuracy: 0.6177
Epoch 2/150
316/316 [==============================] - 2s 7ms/step - loss: 1.1534 - accuracy: 0.6741
Epoch 3/150
316/316 [==============================] - 2s 7ms/step - loss: 1.0412 - accuracy: 0.6791
Epoch 4/150
316/316 [==============================] - 2s 7ms/step - loss: 0.9683 - accuracy: 0.6898
Epoch 5/150
316/316 [==============================] - 2s 7ms/step - loss: 0.9088 - accuracy: 0.7132
Epoch 6/150
316/316 [==============================] - 2s 7ms/step - loss: 0.8596 - accuracy: 0.7298
Epoch 7/150
316/316 [==============================] - 2s 7ms/step - loss: 0.8159 - accuracy: 0.7401
Epoch 8/150
316/316 [==============================] - 2s 7ms/step - loss: 0.7758 - accuracy: 0.7472
Epoch 9/150
316/316 [==

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7ff04030aeb8>>
Epoch 1/150
316/316 [==============================] - 2s 7ms/step - loss: 1.4389 - accuracy: 0.5674
Epoch 2/150
316/316 [==============================] - 2s 7ms/step - loss: 1.1467 - accuracy: 0.6749
Epoch 3/150
316/316 [==============================] - 2s 7ms/step - loss: 1.0272 - accuracy: 0.6827
Epoch 4/150
316/316 [==============================] - 2s 7ms/step - loss: 0.9506 - accuracy: 0.6955
Epoch 5/150
316/316 [==============================] - 2s 7ms/step - loss: 0.8878 - accuracy: 0.7165
Epoch 6/150
316/316 [==============================] - 2s 7ms/step - loss: 0.8368 - accuracy: 0.7328
Epoch 7/150
316/316 [==============================] - 2s 7ms/step - loss: 0.7934 - accuracy: 0.7454
Epoch 8/150
316/316 [==============================] - 2s 7ms/step - loss: 0.7545 - accuracy: 0.7552
Epoch 9/150
316/316 [==============================] - 2s 7ms/step - loss: 0.7216 - a

In [0]:
Run_David_Burns('data/FNOW/MHEALTH.npz')

David M Burns et al. 2018 data/FNOW/MHEALTH.npz
(1335, 250, 23)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef76521940>>
Epoch 1/150
19/19 [==============================] - 0s 15ms/step - loss: 2.4579 - accuracy: 0.0900
Epoch 2/150
19/19 [==============================] - 0s 12ms/step - loss: 2.3997 - accuracy: 0.0917
Epoch 3/150
19/19 [==============================] - 0s 12ms/step - loss: 2.3574 - accuracy: 0.1183
Epoch 4/150
19/19 [==============================] - 0s 12ms/step - loss: 2.3264 - accuracy: 0.1833
Epoch 5/150
19/19 [==============================] - 0s 11ms/step - loss: 2.3000 - accuracy: 0.2383
Epoch 6/150
19/19 [==============================] - 0s 12ms/step - loss: 2.2787 - accuracy: 0.3033
Epoch 7/150
19/19 [==============================] - 0s 12ms/step - loss: 2.2552 - accuracy: 0.3692
Epoch 8/150
19/19 [==============================] - 0s 11ms/step - loss: 2.2378 - accuracy: 0.4142
Epoch 9/150
19/19 [==========

In [0]:
Run_David_Burns('data/FNOW/UTD-MHAD1_1s.npz')

David M Burns et al. 2018 data/FNOW/UTD-MHAD1_1s.npz
(2048, 50, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef74da79e8>>
Epoch 1/150
29/29 [==============================] - 0s 10ms/step - loss: 3.0390 - accuracy: 0.0572
Epoch 2/150
29/29 [==============================] - 0s 5ms/step - loss: 3.0306 - accuracy: 0.0681
Epoch 3/150
29/29 [==============================] - 0s 6ms/step - loss: 3.0235 - accuracy: 0.0850
Epoch 4/150
29/29 [==============================] - 0s 5ms/step - loss: 3.0160 - accuracy: 0.0943
Epoch 5/150
29/29 [==============================] - 0s 5ms/step - loss: 3.0097 - accuracy: 0.1008
Epoch 6/150
29/29 [==============================] - 0s 5ms/step - loss: 3.0044 - accuracy: 0.1128
Epoch 7/150
29/29 [==============================] - 0s 5ms/step - loss: 2.9976 - accuracy: 0.1106
Epoch 8/150
29/29 [==============================] - 0s 5ms/step - loss: 2.9929 - accuracy: 0.1313
Epoch 9/150
29/29 [==============

In [0]:
Run_David_Burns('data/FNOW/UTD-MHAD2_1s.npz')

David M Burns et al. 2018 data/FNOW/UTD-MHAD2_1s.npz
(616, 50, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef74e79c18>>
Epoch 1/150
9/9 [==============================] - 0s 21ms/step - loss: 1.7856 - accuracy: 0.2441
Epoch 2/150
9/9 [==============================] - 0s 5ms/step - loss: 1.7549 - accuracy: 0.3002
Epoch 3/150
9/9 [==============================] - 0s 5ms/step - loss: 1.7327 - accuracy: 0.3201
Epoch 4/150
9/9 [==============================] - 0s 5ms/step - loss: 1.7118 - accuracy: 0.3797
Epoch 5/150
9/9 [==============================] - 0s 5ms/step - loss: 1.6913 - accuracy: 0.4213
Epoch 6/150
9/9 [==============================] - 0s 5ms/step - loss: 1.6758 - accuracy: 0.4340
Epoch 7/150
9/9 [==============================] - 0s 5ms/step - loss: 1.6606 - accuracy: 0.4629
Epoch 8/150
9/9 [==============================] - 0s 5ms/step - loss: 1.6492 - accuracy: 0.4756
Epoch 9/150
9/9 [==============================] -

In [0]:
Run_David_Burns('data/FNOW/WHARF.npz')

David M Burns et al. 2018 data/FNOW/WHARF.npz
(2146, 160, 3)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef743e3d30>>
Epoch 1/150
31/31 [==============================] - 0s 9ms/step - loss: 2.4115 - accuracy: 0.2523
Epoch 2/150
31/31 [==============================] - 0s 9ms/step - loss: 2.3087 - accuracy: 0.3484
Epoch 3/150
31/31 [==============================] - 0s 8ms/step - loss: 2.2416 - accuracy: 0.3868
Epoch 4/150
31/31 [==============================] - 0s 8ms/step - loss: 2.1991 - accuracy: 0.4045
Epoch 5/150
31/31 [==============================] - 0s 8ms/step - loss: 2.1683 - accuracy: 0.4263
Epoch 6/150
31/31 [==============================] - 0s 8ms/step - loss: 2.1355 - accuracy: 0.4278
Epoch 7/150
31/31 [==============================] - 0s 8ms/step - loss: 2.1171 - accuracy: 0.4450
Epoch 8/150
31/31 [==============================] - 0s 8ms/step - loss: 2.0996 - accuracy: 0.4491
Epoch 9/150
31/31 [=====================

In [0]:
Run_David_Burns('data/FNOW/WISDM.npz')

David M Burns et al. 2018 data/FNOW/WISDM.npz
(10516, 100, 3)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7ff04055ee48>>
Epoch 1/150
148/148 [==============================] - 1s 7ms/step - loss: 1.5768 - accuracy: 0.5307
Epoch 2/150
148/148 [==============================] - 1s 7ms/step - loss: 1.3364 - accuracy: 0.6358
Epoch 3/150
148/148 [==============================] - 1s 7ms/step - loss: 1.2190 - accuracy: 0.6635
Epoch 4/150
148/148 [==============================] - 1s 7ms/step - loss: 1.1446 - accuracy: 0.6721
Epoch 5/150
148/148 [==============================] - 1s 7ms/step - loss: 1.0883 - accuracy: 0.6758
Epoch 6/150
148/148 [==============================] - 1s 7ms/step - loss: 1.0461 - accuracy: 0.6772
Epoch 7/150
148/148 [==============================] - 1s 7ms/step - loss: 1.0106 - accuracy: 0.6788
Epoch 8/150
148/148 [==============================] - 1s 7ms/step - loss: 0.9760 - accuracy: 0.6844
Epoch 9/150
148/148 [==

In [0]:
Run_David_Burns('data/LOTO/MHEALTH.npz')

David M Burns et al. 2018 data/LOTO/MHEALTH.npz
(2555, 250, 23)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef71f3bef0>>
Epoch 1/150
36/36 [==============================] - 1s 14ms/step - loss: 2.4331 - accuracy: 0.1309
Epoch 2/150
36/36 [==============================] - 0s 12ms/step - loss: 2.3540 - accuracy: 0.2131
Epoch 3/150
36/36 [==============================] - 0s 12ms/step - loss: 2.2975 - accuracy: 0.2775
Epoch 4/150
36/36 [==============================] - 0s 12ms/step - loss: 2.2517 - accuracy: 0.3823
Epoch 5/150
36/36 [==============================] - 0s 12ms/step - loss: 2.2160 - accuracy: 0.4793
Epoch 6/150
36/36 [==============================] - 0s 12ms/step - loss: 2.1840 - accuracy: 0.5816
Epoch 7/150
36/36 [==============================] - 0s 12ms/step - loss: 2.1552 - accuracy: 0.6390
Epoch 8/150
36/36 [==============================] - 0s 12ms/step - loss: 2.1296 - accuracy: 0.6646
Epoch 9/150
36/36 [==========

In [0]:
Run_David_Burns('data/LOTO/UTD-MHAD1_1s.npz')

David M Burns et al. 2018 data/LOTO/UTD-MHAD1_1s.npz
(3771, 50, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef71dd84e0>>
Epoch 1/150
52/52 [==============================] - 0s 5ms/step - loss: 3.0442 - accuracy: 0.0638
Epoch 2/150
52/52 [==============================] - 0s 6ms/step - loss: 3.0276 - accuracy: 0.0747
Epoch 3/150
52/52 [==============================] - 0s 5ms/step - loss: 3.0147 - accuracy: 0.0849
Epoch 4/150
52/52 [==============================] - 0s 5ms/step - loss: 3.0028 - accuracy: 0.0949
Epoch 5/150
52/52 [==============================] - 0s 6ms/step - loss: 2.9936 - accuracy: 0.1055
Epoch 6/150
52/52 [==============================] - 0s 5ms/step - loss: 2.9846 - accuracy: 0.1115
Epoch 7/150
52/52 [==============================] - 0s 5ms/step - loss: 2.9764 - accuracy: 0.1221
Epoch 8/150
52/52 [==============================] - 0s 5ms/step - loss: 2.9686 - accuracy: 0.1230
Epoch 9/150
52/52 [===============

In [0]:
Run_David_Burns('data/LOTO/UTD-MHAD2_1s.npz')

David M Burns et al. 2018 data/LOTO/UTD-MHAD2_1s.npz
(1137, 50, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fee6df76978>>
Epoch 1/150
16/16 [==============================] - 0s 10ms/step - loss: 1.7733 - accuracy: 0.2294
Epoch 2/150
16/16 [==============================] - 0s 5ms/step - loss: 1.7361 - accuracy: 0.3547
Epoch 3/150
16/16 [==============================] - 0s 5ms/step - loss: 1.7056 - accuracy: 0.4414
Epoch 4/150
16/16 [==============================] - 0s 5ms/step - loss: 1.6846 - accuracy: 0.4567
Epoch 5/150
16/16 [==============================] - 0s 6ms/step - loss: 1.6610 - accuracy: 0.4669
Epoch 6/150
16/16 [==============================] - 0s 5ms/step - loss: 1.6459 - accuracy: 0.4699
Epoch 7/150
16/16 [==============================] - 0s 5ms/step - loss: 1.6297 - accuracy: 0.4832
Epoch 8/150
16/16 [==============================] - 0s 5ms/step - loss: 1.6178 - accuracy: 0.4760
Epoch 9/150
16/16 [==============

In [0]:
Run_David_Burns('data/LOTO/WHARF.npz')

David M Burns et al. 2018 data/LOTO/WHARF.npz
(3871, 160, 3)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef7019aef0>>
Epoch 1/150
55/55 [==============================] - 0s 9ms/step - loss: 2.3359 - accuracy: 0.3923
Epoch 2/150
55/55 [==============================] - 0s 8ms/step - loss: 2.1934 - accuracy: 0.4729
Epoch 3/150
55/55 [==============================] - 0s 8ms/step - loss: 2.1191 - accuracy: 0.5006
Epoch 4/150
55/55 [==============================] - 0s 8ms/step - loss: 2.0663 - accuracy: 0.5144
Epoch 5/150
55/55 [==============================] - 0s 8ms/step - loss: 2.0288 - accuracy: 0.5271
Epoch 6/150
55/55 [==============================] - 0s 8ms/step - loss: 1.9959 - accuracy: 0.5314
Epoch 7/150
55/55 [==============================] - 0s 8ms/step - loss: 1.9666 - accuracy: 0.5302
Epoch 8/150
55/55 [==============================] - 0s 8ms/step - loss: 1.9471 - accuracy: 0.5320
Epoch 9/150
55/55 [=====================

In [0]:
Run_David_Burns('data/LOTO/USCHAD.npz')


David M Burns et al. 2018 data/LOTO/USCHAD.npz
(9824, 500, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef6eb1e5f8>>
Epoch 1/150
138/138 [==============================] - 3s 19ms/step - loss: 2.3282 - accuracy: 0.2501
Epoch 2/150
138/138 [==============================] - 3s 19ms/step - loss: 2.2118 - accuracy: 0.2924
Epoch 3/150
138/138 [==============================] - 3s 19ms/step - loss: 2.1469 - accuracy: 0.2983
Epoch 4/150
138/138 [==============================] - 3s 19ms/step - loss: 2.0994 - accuracy: 0.3031
Epoch 5/150
138/138 [==============================] - 3s 19ms/step - loss: 2.0580 - accuracy: 0.3114
Epoch 6/150
138/138 [==============================] - 3s 19ms/step - loss: 2.0236 - accuracy: 0.3188
Epoch 7/150
138/138 [==============================] - 3s 19ms/step - loss: 1.9924 - accuracy: 0.3260
Epoch 8/150
138/138 [==============================] - 3s 19ms/step - loss: 1.9628 - accuracy: 0.3308
Epoch 9/150
138

In [0]:
Run_David_Burns('data/SNOW/MHEALTH.npz')

David M Burns et al. 2018 data/SNOW/MHEALTH.npz
(2555, 250, 23)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef6eab9320>>
Epoch 1/150
36/36 [==============================] - 1s 14ms/step - loss: 2.4549 - accuracy: 0.1119
Epoch 2/150
36/36 [==============================] - 0s 12ms/step - loss: 2.3597 - accuracy: 0.1820
Epoch 3/150
36/36 [==============================] - 0s 12ms/step - loss: 2.3043 - accuracy: 0.2865
Epoch 4/150
36/36 [==============================] - 0s 12ms/step - loss: 2.2637 - accuracy: 0.4362
Epoch 5/150
36/36 [==============================] - 0s 12ms/step - loss: 2.2302 - accuracy: 0.5734
Epoch 6/150
36/36 [==============================] - 0s 12ms/step - loss: 2.2014 - accuracy: 0.6326
Epoch 7/150
36/36 [==============================] - 0s 12ms/step - loss: 2.1739 - accuracy: 0.6892
Epoch 8/150
36/36 [==============================] - 0s 12ms/step - loss: 2.1518 - accuracy: 0.7148
Epoch 9/150
36/36 [==========

In [26]:
Run_David_Burns('data/SNOW/USCHAD.npz')

David M Burns et al. 2018 data/SNOW/USCHAD.npz
(9824, 500, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef7172ada0>>
Epoch 1/150
139/139 [==============================] - 3s 19ms/step - loss: 2.3397 - accuracy: 0.2790
Epoch 2/150
139/139 [==============================] - 3s 19ms/step - loss: 2.2278 - accuracy: 0.2903
Epoch 3/150
139/139 [==============================] - 3s 19ms/step - loss: 2.1568 - accuracy: 0.2948
Epoch 4/150
139/139 [==============================] - 3s 19ms/step - loss: 2.1056 - accuracy: 0.3036
Epoch 5/150
139/139 [==============================] - 3s 19ms/step - loss: 2.0619 - accuracy: 0.3111
Epoch 6/150
139/139 [==============================] - 3s 19ms/step - loss: 2.0251 - accuracy: 0.3154
Epoch 7/150
139/139 [==============================] - 3s 19ms/step - loss: 1.9920 - accuracy: 0.3248
Epoch 8/150
139/139 [==============================] - 3s 19ms/step - loss: 1.9651 - accuracy: 0.3297
Epoch 9/150
139

In [27]:
Run_David_Burns('data/SNOW/UTD-MHAD1_1s.npz')

David M Burns et al. 2018 data/SNOW/UTD-MHAD1_1s.npz
(3771, 50, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fef7170f5c0>>
Epoch 1/150
53/53 [==============================] - 0s 8ms/step - loss: 3.0509 - accuracy: 0.0446
Epoch 2/150
53/53 [==============================] - 0s 6ms/step - loss: 3.0343 - accuracy: 0.0600
Epoch 3/150
53/53 [==============================] - 0s 6ms/step - loss: 3.0199 - accuracy: 0.0777
Epoch 4/150
53/53 [==============================] - 0s 5ms/step - loss: 3.0063 - accuracy: 0.0924
Epoch 5/150
53/53 [==============================] - 0s 7ms/step - loss: 2.9947 - accuracy: 0.1081
Epoch 6/150
53/53 [==============================] - 0s 6ms/step - loss: 2.9855 - accuracy: 0.1282
Epoch 7/150
53/53 [==============================] - 0s 5ms/step - loss: 2.9763 - accuracy: 0.1385
Epoch 8/150
53/53 [==============================] - 0s 5ms/step - loss: 2.9683 - accuracy: 0.1429
Epoch 9/150
53/53 [===============

In [28]:
Run_David_Burns('data/SNOW/UTD-MHAD2_1s.npz')

David M Burns et al. 2018 data/SNOW/UTD-MHAD2_1s.npz
(1137, 50, 6)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fee6b957da0>>
Epoch 1/150
16/16 [==============================] - 0s 5ms/step - loss: 1.7678 - accuracy: 0.2807
Epoch 2/150
16/16 [==============================] - 0s 5ms/step - loss: 1.7341 - accuracy: 0.3788
Epoch 3/150
16/16 [==============================] - 0s 5ms/step - loss: 1.7077 - accuracy: 0.4014
Epoch 4/150
16/16 [==============================] - 0s 5ms/step - loss: 1.6837 - accuracy: 0.4289
Epoch 5/150
16/16 [==============================] - 0s 6ms/step - loss: 1.6651 - accuracy: 0.4387
Epoch 6/150
16/16 [==============================] - 0s 5ms/step - loss: 1.6457 - accuracy: 0.4495
Epoch 7/150
16/16 [==============================] - 0s 5ms/step - loss: 1.6293 - accuracy: 0.4622
Epoch 8/150
16/16 [==============================] - 0s 6ms/step - loss: 1.6157 - accuracy: 0.4593
Epoch 9/150
16/16 [===============

In [29]:
Run_David_Burns('data/SNOW/WHARF.npz')

David M Burns et al. 2018 data/SNOW/WHARF.npz
(3880, 160, 3)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fee6d7ac898>>
Epoch 1/150
55/55 [==============================] - 1s 10ms/step - loss: 2.3742 - accuracy: 0.2353
Epoch 2/150
55/55 [==============================] - 0s 8ms/step - loss: 2.2165 - accuracy: 0.4494
Epoch 3/150
55/55 [==============================] - 0s 9ms/step - loss: 2.1358 - accuracy: 0.5248
Epoch 4/150
55/55 [==============================] - 0s 8ms/step - loss: 2.0795 - accuracy: 0.5475
Epoch 5/150
55/55 [==============================] - 0s 9ms/step - loss: 2.0391 - accuracy: 0.5527
Epoch 6/150
55/55 [==============================] - 0s 8ms/step - loss: 2.0027 - accuracy: 0.5555
Epoch 7/150
55/55 [==============================] - 0s 8ms/step - loss: 1.9716 - accuracy: 0.5492
Epoch 8/150
55/55 [==============================] - 0s 9ms/step - loss: 1.9478 - accuracy: 0.5558
Epoch 9/150
55/55 [====================

In [30]:
Run_David_Burns('data/SNOW/WISDM.npz')

David M Burns et al. 2018 data/SNOW/WISDM.npz
(20846, 100, 3)
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7fee6dcca6a0>>
Epoch 1/150
294/294 [==============================] - 2s 7ms/step - loss: 1.4595 - accuracy: 0.6087
Epoch 2/150
294/294 [==============================] - 2s 7ms/step - loss: 1.1832 - accuracy: 0.6690
Epoch 3/150
294/294 [==============================] - 2s 7ms/step - loss: 1.0644 - accuracy: 0.6791
Epoch 4/150
294/294 [==============================] - 2s 7ms/step - loss: 0.9885 - accuracy: 0.6826
Epoch 5/150
294/294 [==============================] - 2s 7ms/step - loss: 0.9284 - accuracy: 0.6952
Epoch 6/150
294/294 [==============================] - 2s 7ms/step - loss: 0.8796 - accuracy: 0.7135
Epoch 7/150
294/294 [==============================] - 2s 7ms/step - loss: 0.8351 - accuracy: 0.7312
Epoch 8/150
294/294 [==============================] - 2s 7ms/step - loss: 0.7950 - accuracy: 0.7420
Epoch 9/150
294/294 [==